In [100]:
import pandas as pd

In [101]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Below all the sites in London are imported

In [102]:
sites = pd.read_table('/content/drive/MyDrive/ULEZ/sites.tsv')
sites.head(10)

,LocalAuthorityName,SiteCode,SiteName,SiteType,DateClosed,DateOpened,Latitude,Longitude
0,Richmond,TD0,"- National Physical Laboratory, Teddington",Suburban,2018-01-01 00:00:00,1996-08-08 00:00:00,51.424304,-0.345715
1,Barking and Dagenham,BG3,Barking and Dagenham - North Street,Kerbside,2011-05-25 00:00:00,2007-03-16 00:00:00,51.540444,0.074418
2,Barking and Dagenham,BG1,Barking and Dagenham - Rush Green,Suburban,NaN,1999-11-02 00:00:00,51.563752,0.177891
3,Barking and Dagenham,BG2,Barking and Dagenham - Scrattons Farm,Suburban,NaN,1999-10-17 00:00:00,51.529389,0.132857
4,Barnet,BN2,Barnet - Finchley,Urban Background,2012-04-20 00:00:00,2000-08-09 13:00:00,51.591901,-0.205992
5,Barnet,BN3,Barnet - Strawberry Vale,Urban Background,2002-05-15 00:00:00,2000-08-14 14:00:00,51.600885,-0.172298
6,Barnet,BN1,Barnet - Tally Ho Corner,Kerbside,2012-04-20 00:00:00,1998-12-20 12:00:00,51.614675,-0.176607
7,Bexley,BX5,Bexley - Bedonwell,Suburban,2004-04-08 00:00:00,1999-10-06 13:00:00,51.478802,0.133874
8,Bexley,BX2,Bexley - Belvedere,Suburban,NaN,1998-01-24 15:00:00,51.490610,0.158914
9,Bexley,BX0,Bexley - Belvedere FDMS,Suburban,2022-01-28 00:00:00,2004-04-28 00:00:00,51.490610,0.158914


In [103]:
sites.shape

(252, 8)

In [104]:
sites.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   LocalAuthorityName  252 non-null    object 
 1   SiteCode            252 non-null    object 
 2   SiteName            252 non-null    object 
 3   SiteType            252 non-null    object 
 4   DateClosed          144 non-null    object 
 5   DateOpened          252 non-null    object 
 6   Latitude            250 non-null    float64
 7   Longitude           250 non-null    float64
dtypes: float64(2), object(6)
memory usage: 15.9+ KB


In [105]:
# making all columns lower case
sites.columns = sites.columns.str.lower()

# renaming columns
sites = sites.rename(columns={'localauthorityname': 'borough', 'sitecode': 'code', 'sitename': 'name', 'sitetype': 'type', 'dateclosed': 'date_closed', 'dateopened': 'date_opened'})

# rearranging the columns in convinient way
columns = sites.columns.tolist()
new_order_columns = ['name', 'code', 'borough', 'type', 'latitude', 'longitude', 'date_opened', 'date_closed']
sites = sites[new_order_columns]

# correcting '- National Physical Laboratory, Teddington' to 'Teddington - National Physical Laboratory'
sites.loc[sites['name'] == '- National Physical Laboratory, Teddington', 'name'] = 'Teddington - National Physical Laboratory'

# separating borough from site name, keeping site name only
sites['name'] = sites.name.apply(lambda x: x.split('-')[-1])

# remove whitespace from names in name column
sites['name'] = sites['name'].str.strip()

# removing the time from date_closed and date_opened
sites['date_opened'] = pd.to_datetime(sites['date_opened'])
sites['date_closed'] = pd.to_datetime(sites['date_closed'])

sites['date_opened'] = sites['date_opened'].dt.date
sites['date_closed'] = sites['date_closed'].dt.date

# changing nan in closed_date column (still open) to 01/08/2023 for convinience
sites['date_closed'].fillna(pd.to_datetime('2023-08-01'), inplace = True)

sites.head()

,name,code,borough,type,latitude,longitude,date_opened,date_closed
0,National Physical Laboratory,TD0,Richmond,Suburban,51.424304,-0.345715,1996-08-08,2018-01-01
1,North Street,BG3,Barking and Dagenham,Kerbside,51.540444,0.074418,2007-03-16,2011-05-25
2,Rush Green,BG1,Barking and Dagenham,Suburban,51.563752,0.177891,1999-11-02,2023-08-01 00:00:00
3,Scrattons Farm,BG2,Barking and Dagenham,Suburban,51.529389,0.132857,1999-10-17,2023-08-01 00:00:00
4,Finchley,BN2,Barnet,Urban Background,51.591901,-0.205992,2000-08-09,2012-04-20


In [106]:
# checking how many sites are in each borough
sites.borough.value_counts()

Westminster               22
Bexley                    21
Ealing                    14
Greenwich                 12
Wandsworth                11
Southwark                 10
Kensington and Chelsea    10
Croydon                    9
Hounslow                   9
Sutton                     9
Brent                      8
Camden                     8
City of London             8
Kingston                   7
Lewisham                   7
Enfield                    7
Redbridge                  6
Newham                     6
Hillingdon                 6
Merton                     6
Tower Hamlets              6
Lambeth                    6
Hackney                    6
Waltham Forest             5
Richmond                   5
Islington                  5
Haringey                   4
Hammersmith and Fulham     4
Bromley                    4
Barking and Dagenham       3
Barnet                     3
Havering                   3
Harrow                     2
Name: borough, dtype: int64

# **INITIAL ULEZ**

We need to choose sites wich are relevant to us:

**Initial ULEZ**

*Boroughs:*

1.   Camden
2.   City of London
3.   Hackney
4.   Islington
5.   Southwark
6.   Tower Hamlets
7.   Westminster



In [107]:
# choosing sites for initial ULEZ
initial_ulez_boroughs = ['Camden', 'City of London', 'Hackney', 'Islington', 'Southwark', 'Tower Hamlets', 'Westminster']

in_ulez_sites = sites[sites['borough'].isin(initial_ulez_boroughs)]

in_ulez_sites.borough.value_counts().sort_index()

Camden             8
City of London     8
Hackney            6
Islington          5
Southwark         10
Tower Hamlets      6
Westminster       22
Name: borough, dtype: int64

In [108]:
in_ulez_sites.shape

(65, 8)

We should find sites with:

latitude between 51.485486773973 and 51.526454000000

longitude between -0.164338000000 and -0.070078971000

These are approximate coordinates of borders of the initial ULEZ.


In [109]:
in_ulez_sites = in_ulez_sites.loc[(in_ulez_sites['latitude'] > 51.485486773973) & (in_ulez_sites['latitude'] < 51.526454000000)]
in_ulez_sites = in_ulez_sites.loc[(in_ulez_sites['longitude'] > -0.164338000000) & (in_ulez_sites['longitude'] < -0.070078971000)]
in_ulez_sites.shape

(39, 8)

In [110]:
# dropping sites with date_closed before 01/01/2023
# dropping sites with date_opened after 01/01/2017
open_code = pd.to_datetime('2017-01-01')
close_date = pd.to_datetime('2023-01-01')

in_ulez_sites = in_ulez_sites.loc[in_ulez_sites['date_opened'] < open_code]
in_ulez_sites = in_ulez_sites.loc[in_ulez_sites['date_closed'] > close_date]

# resetting the index
in_ulez_sites = in_ulez_sites.reset_index(drop=True)

in_ulez_sites.borough.value_counts().sort_index()

Camden            2
City of London    4
Southwark         1
Westminster       6
Name: borough, dtype: int64

In [111]:
in_ulez_sites.shape

(13, 8)

In [112]:
in_ulez_sites.head(14)

,name,code,borough,type,latitude,longitude,date_opened,date_closed
0,Bloomsbury,BL0,Camden,Urban Background,51.522287,-0.125848,1992-01-23,2023-08-01 00:00:00
1,Holborn (Bee Midtown),IM1,Camden,Kerbside,51.517368,-0.120195,2014-02-06,2023-08-01 00:00:00
2,Beech Street,CT4,City of London,Roadside,51.520225,-0.096106,2003-01-01,2023-07-03
3,Farringdon Street,CT2,City of London,Kerbside,51.514525,-0.104516,2003-01-01,2023-07-03
4,The Aldgate School,CT3,City of London,Urban Background,51.513847,-0.077766,2003-01-01,2023-06-30
5,Walbrook Wharf,CT6,City of London,Roadside,51.510499,-0.091634,2007-04-01,2023-01-09
6,BAM,MR8,Westminster,Kerbside,51.522509,-0.154622,2012-01-11,2023-08-01 00:00:00
7,Elephant and Castle,SK6,Southwark,Urban Background,51.493156,-0.101527,2013-01-23,2023-08-01 00:00:00
8,Covent Garden,WM5,Westminster,Urban Background,51.511977,-0.121627,2009-07-10,2023-08-01 00:00:00
9,Horseferry Road,WM0,Westminster,Urban Background,51.494681,-0.131938,2001-07-17,2023-08-01 00:00:00


In [113]:
# creating dictionaries with code and columns name, borough, type, latitude, longitude
name_d = dict(zip(in_ulez_sites.code, in_ulez_sites.name))
borough_d = dict(zip(in_ulez_sites.code, in_ulez_sites.borough))
type_d = dict(zip(in_ulez_sites.code, in_ulez_sites.type))
latitude_d = dict(zip(in_ulez_sites.code, in_ulez_sites.latitude))
longitude_d = dict(zip(in_ulez_sites.code, in_ulez_sites.longitude))
print(name_d)
print(borough_d)
print(type_d)
print(latitude_d)
print(longitude_d)

{'BL0': 'Bloomsbury', 'IM1': 'Holborn (Bee Midtown)', 'CT4': 'Beech Street', 'CT2': 'Farringdon Street', 'CT3': 'The Aldgate School', 'CT6': 'Walbrook Wharf', 'MR8': 'BAM', 'SK6': 'Elephant and Castle', 'WM5': 'Covent Garden', 'WM0': 'Horseferry Road', 'MY1': 'Marylebone Road', 'WM6': 'Oxford Street', 'NB1': 'Strand (Northbank BID)'}
{'BL0': 'Camden', 'IM1': 'Camden', 'CT4': 'City of London', 'CT2': 'City of London', 'CT3': 'City of London', 'CT6': 'City of London', 'MR8': 'Westminster', 'SK6': 'Southwark', 'WM5': 'Westminster', 'WM0': 'Westminster', 'MY1': 'Westminster', 'WM6': 'Westminster', 'NB1': 'Westminster'}
{'BL0': 'Urban Background', 'IM1': 'Kerbside', 'CT4': 'Roadside', 'CT2': 'Kerbside', 'CT3': 'Urban Background', 'CT6': 'Roadside', 'MR8': 'Kerbside', 'SK6': 'Urban Background', 'WM5': 'Urban Background', 'WM0': 'Urban Background', 'MY1': 'Kerbside', 'WM6': 'Kerbside', 'NB1': 'Roadside'}
{'BL0': 51.522287, 'IM1': 51.5173675146177, 'CT4': 51.520225273171, 'CT2': 51.51452533623

Now we need to get the names and the codes of the sites. We will use them to pull data about NO2 from London Air using API.

In [114]:
# getting sites' codes
in_ulez_sites_codes = in_ulez_sites.code
print(type(in_ulez_sites_codes))

# converting pandas series to list
in_ulez_sites_codes = in_ulez_sites_codes.to_list()
print(in_ulez_sites_codes)
print(type(in_ulez_sites_codes))

<class 'pandas.core.series.Series'>
['BL0', 'IM1', 'CT4', 'CT2', 'CT3', 'CT6', 'MR8', 'SK6', 'WM5', 'WM0', 'MY1', 'WM6', 'NB1']
<class 'list'>


Below the Nitrogen Dioxide (NO2) data from 01/01/2017 to 01/01/2023 for the initial ulez is imported.

In [115]:
in_ulez_data = pd.read_csv('/content/drive/MyDrive/ULEZ/no2_in_ulez.csv')
in_ulez_data.head()

,Site,Species,ReadingDateTime,Value,Units,Provisional or Ratified
0,BL0,NO2,01/01/2017 00:00,24.5,ug m-3,R
1,BL0,NO2,02/01/2017 00:00,34.5,ug m-3,R
2,BL0,NO2,03/01/2017 00:00,46.9,ug m-3,R
3,BL0,NO2,04/01/2017 00:00,45.8,ug m-3,R
4,BL0,NO2,05/01/2017 00:00,59.6,ug m-3,R


In [116]:
in_ulez_data.shape

(28483, 6)

In [117]:
# adding columns name, borough, type	latitude	longitude to in_ulez_data
in_ulez_data['name'] = in_ulez_data['Site'].map(name_d)
in_ulez_data['borough'] = in_ulez_data['Site'].map(borough_d)
in_ulez_data['type'] = in_ulez_data['Site'].map(type_d)
in_ulez_data['latitude'] = in_ulez_data['Site'].map(latitude_d)
in_ulez_data['longitude'] = in_ulez_data['Site'].map(longitude_d)

In [118]:
in_ulez_data.head()

,Site,Species,ReadingDateTime,Value,Units,Provisional or Ratified,name,borough,type,latitude,longitude
0,BL0,NO2,01/01/2017 00:00,24.5,ug m-3,R,Bloomsbury,Camden,Urban Background,51.522287,-0.125848
1,BL0,NO2,02/01/2017 00:00,34.5,ug m-3,R,Bloomsbury,Camden,Urban Background,51.522287,-0.125848
2,BL0,NO2,03/01/2017 00:00,46.9,ug m-3,R,Bloomsbury,Camden,Urban Background,51.522287,-0.125848
3,BL0,NO2,04/01/2017 00:00,45.8,ug m-3,R,Bloomsbury,Camden,Urban Background,51.522287,-0.125848
4,BL0,NO2,05/01/2017 00:00,59.6,ug m-3,R,Bloomsbury,Camden,Urban Background,51.522287,-0.125848


In [119]:
# making all columns lower case
in_ulez_data.columns = in_ulez_data.columns.str.lower()

# renaming columns
in_ulez_data = in_ulez_data.rename(columns={'readingdatetime': 'reading_date', 'provisional or ratified': 'prov_rat', 'site': 'code'})

# rearranging the columns in convinient way
columns = in_ulez_data.columns.tolist()
new_order_columns = ['name', 'borough', 'code', 'latitude', 'longitude', 'type', 'species', 'reading_date', 'value', 'units', 'prov_rat']
in_ulez_data = in_ulez_data[new_order_columns]

# removing the time from date_closed and date_opened
in_ulez_data['reading_date'] = pd.to_datetime(in_ulez_data['reading_date'])
in_ulez_data['reading_date'] = in_ulez_data['reading_date'].dt.date

in_ulez_data.head()

,name,borough,code,latitude,longitude,type,species,reading_date,value,units,prov_rat
0,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-01-01,24.5,ug m-3,R
1,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-02-01,34.5,ug m-3,R
2,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-03-01,46.9,ug m-3,R
3,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-04-01,45.8,ug m-3,R
4,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-05-01,59.6,ug m-3,R


In [120]:
in_ulez_data.shape

(28483, 11)

In [121]:
in_ulez_data['ID'] = in_ulez_data['reading_date'].astype(str) + in_ulez_data['latitude'].astype(str) + in_ulez_data['longitude'].astype(str)

in_ulez_data = in_ulez_data.set_index('ID')

in_ulez_data.head()

,name,borough,code,latitude,longitude,type,species,reading_date,value,units,prov_rat
ID,,,,,,,,,,,
2017-01-0151.522287-0.125848,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-01-01,24.5,ug m-3,R
2017-02-0151.522287-0.125848,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-02-01,34.5,ug m-3,R
2017-03-0151.522287-0.125848,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-03-01,46.9,ug m-3,R
2017-04-0151.522287-0.125848,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-04-01,45.8,ug m-3,R
2017-05-0151.522287-0.125848,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-05-01,59.6,ug m-3,R


In [122]:
in_ulez_data.shape

(28483, 11)

# Collecting weather data

In [123]:
!pip install meteostat

In [124]:
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily

In [125]:
# use columns from in_ulez_data to create dictionary of coordinates
columns_to_include = ['latitude', 'longitude']

# Convert selected columns to a dictionary
location_sites = in_ulez_sites[columns_to_include].to_dict('index')

location_sites

{0: {'latitude': 51.522287, 'longitude': -0.125848},
 1: {'latitude': 51.5173675146177, 'longitude': -0.1201947113171},
 2: {'latitude': 51.520225273171, 'longitude': -0.0961060474176381},
 3: {'latitude': 51.5145253362314, 'longitude': -0.104515626337876},
 4: {'latitude': 51.513847178423, 'longitude': -0.077765681752},
 5: {'latitude': 51.510499, 'longitude': -0.091634},
 6: {'latitude': 51.5225093597419, 'longitude': -0.154622154656843},
 7: {'latitude': 51.4931557048667, 'longitude': -0.101527038274853},
 8: {'latitude': 51.5119769825798, 'longitude': -0.12162720332856},
 9: {'latitude': 51.494681, 'longitude': -0.131938},
 10: {'latitude': 51.52254, 'longitude': -0.15459},
 11: {'latitude': 51.5139287404213, 'longitude': -0.152792701881935},
 12: {'latitude': 51.5119701188277, 'longitude': -0.116713103528546}}

In [126]:
def get_weather(x):
# empty list for data
  weather_data_list = []

# for loop to iterate over the location sites dictionary created in the previous step
  for loc, coordinates in x.items():
    latitude = coordinates['latitude']
    longitude = coordinates['longitude']

# combine the lat and lon and use Point to find the site
    location = Point(latitude, longitude)

# specify date range
    start = datetime(2017, 1, 1)
    end = datetime(2023, 1, 1)

# make the call
    weather_data = Daily(location, start, end).fetch()

# add latitude and longitude columns to the weather data
    weather_data['latitude'] = latitude
    weather_data['longitude'] = longitude

# append the data to the empty list
    weather_data_list.append(weather_data)

# transform the data into a DataFrame
  combined_weather_data = pd.concat(weather_data_list)

  return combined_weather_data

# call the function
weather_df = get_weather(location_sites)

# print data to check
print(weather_df.head())

            tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun  \
time                                                                        
2017-01-01   5.9   4.0   9.4   0.3   NaN  241.0  14.0   NaN  1018.6   NaN   
2017-01-02   3.9   2.0   6.0   6.6   NaN  352.0  11.3   NaN  1028.4   NaN   
2017-01-03   3.3   0.0   6.0   0.0   NaN  250.0  18.6   NaN  1028.0   NaN   
2017-01-04   5.8   3.0   8.0   0.3   NaN  296.0  15.8   NaN  1025.6   NaN   
2017-01-05   3.0   1.0   6.0   0.0   NaN  315.0   5.7   NaN  1034.4   NaN   

             latitude  longitude  
time                              
2017-01-01  51.522287  -0.125848  
2017-01-02  51.522287  -0.125848  
2017-01-03  51.522287  -0.125848  
2017-01-04  51.522287  -0.125848  
2017-01-05  51.522287  -0.125848  


In [127]:
# Optional - Export the DataFrame to a TSV file
# file_path = "weather_data_inital.tsv"
# weather_df.to_csv(file_path, sep='\t', index=False)

# print(f"DataFrame exported to {file_path}")

In [128]:
weather_df.reset_index(inplace=True, level=['time'])

In [129]:
# renaming columns
weather_df = weather_df.rename(columns={'time': 'reading_date'})

print(weather_df.head())

  reading_date  tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun  \
0   2017-01-01   5.9   4.0   9.4   0.3   NaN  241.0  14.0   NaN  1018.6   NaN   
1   2017-01-02   3.9   2.0   6.0   6.6   NaN  352.0  11.3   NaN  1028.4   NaN   
2   2017-01-03   3.3   0.0   6.0   0.0   NaN  250.0  18.6   NaN  1028.0   NaN   
3   2017-01-04   5.8   3.0   8.0   0.3   NaN  296.0  15.8   NaN  1025.6   NaN   
4   2017-01-05   3.0   1.0   6.0   0.0   NaN  315.0   5.7   NaN  1034.4   NaN   

    latitude  longitude  
0  51.522287  -0.125848  
1  51.522287  -0.125848  
2  51.522287  -0.125848  
3  51.522287  -0.125848  
4  51.522287  -0.125848  


In [130]:
# rearranging the columns to position date and lat and lon on left
columns = weather_df.columns.tolist()
print(columns)

['reading_date', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun', 'latitude', 'longitude']


In [131]:
new_order_columns = ['reading_date','latitude', 'longitude','tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun']
weather_df = weather_df[new_order_columns]
print(weather_df.head())

  reading_date   latitude  longitude  tavg  tmin  tmax  prcp  snow   wdir  \
0   2017-01-01  51.522287  -0.125848   5.9   4.0   9.4   0.3   NaN  241.0   
1   2017-01-02  51.522287  -0.125848   3.9   2.0   6.0   6.6   NaN  352.0   
2   2017-01-03  51.522287  -0.125848   3.3   0.0   6.0   0.0   NaN  250.0   
3   2017-01-04  51.522287  -0.125848   5.8   3.0   8.0   0.3   NaN  296.0   
4   2017-01-05  51.522287  -0.125848   3.0   1.0   6.0   0.0   NaN  315.0   

   wspd  wpgt    pres  tsun  
0  14.0   NaN  1018.6   NaN  
1  11.3   NaN  1028.4   NaN  
2  18.6   NaN  1028.0   NaN  
3  15.8   NaN  1025.6   NaN  
4   5.7   NaN  1034.4   NaN  


In [132]:
# create an ID of a unqiue value by combining date, lon and lat and coverting to string so they concat. Set it as index

weather_df['ID'] = weather_df['reading_date'].astype(str) + weather_df['latitude'].astype(str) + weather_df['longitude'].astype(str)

weather_df = weather_df.set_index('ID')

weather_df.head()

,reading_date,latitude,longitude,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
ID,,,,,,,,,,,,,
2017-01-0151.522287-0.125848,2017-01-01,51.522287,-0.125848,5.9,4.0,9.4,0.3,NaN,241.0,14.0,NaN,1018.6,NaN
2017-01-0251.522287-0.125848,2017-01-02,51.522287,-0.125848,3.9,2.0,6.0,6.6,NaN,352.0,11.3,NaN,1028.4,NaN
2017-01-0351.522287-0.125848,2017-01-03,51.522287,-0.125848,3.3,0.0,6.0,0.0,NaN,250.0,18.6,NaN,1028.0,NaN
2017-01-0451.522287-0.125848,2017-01-04,51.522287,-0.125848,5.8,3.0,8.0,0.3,NaN,296.0,15.8,NaN,1025.6,NaN
2017-01-0551.522287-0.125848,2017-01-05,51.522287,-0.125848,3.0,1.0,6.0,0.0,NaN,315.0,5.7,NaN,1034.4,NaN


In [133]:
weather_df.shape

(28496, 13)

In [134]:
# merge the two dataframes on the ID

merged_data_ulez_1 = pd.merge(in_ulez_data, weather_df, on=['ID'])

merged_data_ulez_1.head()

,name,borough,code,latitude_x,longitude_x,type,species,reading_date_x,value,units,...,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
ID,,,,,,,,,,,,,,,,,,,,,
2017-01-0151.522287-0.125848,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-01-01,24.5,ug m-3,...,5.9,4.0,9.4,0.3,NaN,241.0,14.0,NaN,1018.6,NaN
2017-02-0151.522287-0.125848,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-02-01,34.5,ug m-3,...,10.5,8.0,12.0,3.0,NaN,180.0,13.3,NaN,1006.5,NaN
2017-03-0151.522287-0.125848,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-03-01,46.9,ug m-3,...,8.0,6.0,10.0,1.5,NaN,248.0,20.0,NaN,998.2,NaN
2017-04-0151.522287-0.125848,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-04-01,45.8,ug m-3,...,12.8,11.0,15.0,0.3,NaN,212.0,15.6,NaN,1009.4,NaN
2017-05-0151.522287-0.125848,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-05-01,59.6,ug m-3,...,11.2,10.0,13.0,1.5,NaN,117.0,16.1,NaN,1004.2,NaN


In [135]:
merged_data_ulez_1.shape

(28483, 24)

In [136]:
file_path = "merged_data_ulez_1.csv"

merged_data_ulez_1.to_csv(file_path)
# printed just to double-check that file has been created successfully
print(f"DataFrame exported to {file_path}")

DataFrame exported to merged_data_ulez_1.csv


# Cleaning the merged data

In [137]:
# importing our merged ULEZ dataset
in_ulez_data = pd.read_csv('/content/drive/MyDrive/ULEZ/merged_data_ulez_1.csv')
in_ulez_data.head()

,ID,name,borough,code,latitude_x,longitude_x,type,species,reading_date_x,value,...,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2017-01-0151.522287-0.125848,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-01-01,24.5,...,5.9,4.0,9.4,0.3,NaN,241.0,14.0,NaN,1018.6,NaN
1,2017-02-0151.522287-0.125848,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-02-01,34.5,...,10.5,8.0,12.0,3.0,NaN,180.0,13.3,NaN,1006.5,NaN
2,2017-03-0151.522287-0.125848,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-03-01,46.9,...,8.0,6.0,10.0,1.5,NaN,248.0,20.0,NaN,998.2,NaN
3,2017-04-0151.522287-0.125848,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-04-01,45.8,...,12.8,11.0,15.0,0.3,NaN,212.0,15.6,NaN,1009.4,NaN
4,2017-05-0151.522287-0.125848,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,NO2,2017-05-01,59.6,...,11.2,10.0,13.0,1.5,NaN,117.0,16.1,NaN,1004.2,NaN


In [138]:
in_ulez_data.shape

(28483, 25)

In [139]:
in_ulez_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28483 entries, 0 to 28482
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              28483 non-null  object 
 1   name            28483 non-null  object 
 2   borough         28483 non-null  object 
 3   code            28483 non-null  object 
 4   latitude_x      28483 non-null  float64
 5   longitude_x     28483 non-null  float64
 6   type            28483 non-null  object 
 7   species         28483 non-null  object 
 8   reading_date_x  28483 non-null  object 
 9   value           22450 non-null  float64
 10  units           28483 non-null  object 
 11  prov_rat        28483 non-null  object 
 12  reading_date_y  28483 non-null  object 
 13  latitude_y      28483 non-null  float64
 14  longitude_y     28483 non-null  float64
 15  tavg            28483 non-null  float64
 16  tmin            28418 non-null  float64
 17  tmax            28444 non-null 

In [140]:
# checking all the columns
columns = in_ulez_data.columns.tolist()
columns

# checking if columns latitude_x and latitude_y, longitude_x and longitude_y, reading_date_x and reading_date_y are identical.
print(in_ulez_data['latitude_x'].equals(in_ulez_data['latitude_y']))
print(in_ulez_data['longitude_x'].equals(in_ulez_data['longitude_y']))
print(in_ulez_data['reading_date_x'].equals(in_ulez_data['reading_date_y']))

# the columns above are identical so we are dropping latitude_y, longitude_y and reading_date_y. Also dropping species column, because it has only one value which is NO2,
# ID - it was used for merging, and units, which also has the same value for all rows ug m-3, tsun - no values in there.
in_ulez_data = in_ulez_data.drop(['latitude_y', 'longitude_y', 'reading_date_y', 'species', 'ID', 'units', 'tsun'], axis=1)

# renaming columns
in_ulez_data = in_ulez_data.rename(columns={'name': 'site_name', 'borough': 'site_borough', 'code': 'site_code', 'type': 'site_type', 'latitude_x': 'latitude', 'longitude_x': 'longitude', 'reading_date_x': 'date', 'value': 'NO2_value', 'tavg': 'avg_temperature',
                                            'tmin': 'min_temperature', 'tmax': 'max_temperature', 'prcp': 'precipitation', 'snow': 'snow_depth', 'wdir': 'avg_wind_direction',
                                            'wspd': 'avg_wind_speed', 'wpgt': 'wind_gust', 'pres': 'avg_air_pressure'})

# converting the date format to datetime
in_ulez_data['date'] = pd.to_datetime(in_ulez_data['date'])

in_ulez_data.head()

True
True
True


,site_name,site_borough,site_code,latitude,longitude,site_type,date,NO2_value,prov_rat,avg_temperature,min_temperature,max_temperature,precipitation,snow_depth,avg_wind_direction,avg_wind_speed,wind_gust,avg_air_pressure
0,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,2017-01-01,24.5,R,5.9,4.0,9.4,0.3,NaN,241.0,14.0,NaN,1018.6
1,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,2017-02-01,34.5,R,10.5,8.0,12.0,3.0,NaN,180.0,13.3,NaN,1006.5
2,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,2017-03-01,46.9,R,8.0,6.0,10.0,1.5,NaN,248.0,20.0,NaN,998.2
3,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,2017-04-01,45.8,R,12.8,11.0,15.0,0.3,NaN,212.0,15.6,NaN,1009.4
4,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,2017-05-01,59.6,R,11.2,10.0,13.0,1.5,NaN,117.0,16.1,NaN,1004.2


In [141]:
in_ulez_data.shape

(28483, 18)

In [142]:
in_ulez_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28483 entries, 0 to 28482
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   site_name           28483 non-null  object        
 1   site_borough        28483 non-null  object        
 2   site_code           28483 non-null  object        
 3   latitude            28483 non-null  float64       
 4   longitude           28483 non-null  float64       
 5   site_type           28483 non-null  object        
 6   date                28483 non-null  datetime64[ns]
 7   NO2_value           22450 non-null  float64       
 8   prov_rat            28483 non-null  object        
 9   avg_temperature     28483 non-null  float64       
 10  min_temperature     28418 non-null  float64       
 11  max_temperature     28444 non-null  float64       
 12  precipitation       28431 non-null  float64       
 13  snow_depth          897 non-null    float64   

We have the following columns in our dataset:

* **site_name** - the name of the monitoring site
* **site_borough**  - the name of the borough where monitoring site is
* **site_code** - code of the monitoring site           
* **latitude** - latitude of the monitoring site
* **longitude** - longitude of the monitoring site
* **site_type** - type of the monitoring site
* **date** - date of the collection
* **NO2_value** - average value of Nitrogen Dioxide (NO2), ug m-3
* **prov_rat** - provisional or ratified
* **avg_temperature** - the average air temperature, °C             
* **min_temperature** - the minimum air temperature, °C             
* **max_temperature** - the maximum air temperature, °C             
* **precipitation** - the daily precipitation total, mm             
* **snow_depth** - the snow depth, mm              
* **avg_wind_direction** - the average wind direction, degree (°)              
* **avg_wind_speed** - the average wind speed, km/h             
* **wind_gust** - the peak wind gust, km/h             
* **avg_air_pressure** - the average sea-level air pressure, hPa          

In our analysis we will use NO2 values which are ratified.

In [143]:
# removing all rows where prov_rat = P
in_ulez_data = in_ulez_data[in_ulez_data['prov_rat'] == 'R']
in_ulez_data.prov_rat.value_counts()

R    23006
Name: prov_rat, dtype: int64

Working with Nan values:

In [144]:
# checking all Nan values in the dataset
in_ulez_data.isna().sum()

site_name                 0
site_borough              0
site_code                 0
latitude                  0
longitude                 0
site_type                 0
date                      0
NO2_value              1809
prov_rat                  0
avg_temperature           0
min_temperature          52
max_temperature          33
precipitation            44
snow_depth            22418
avg_wind_direction      115
avg_wind_speed           80
wind_gust              7122
avg_air_pressure         63
dtype: int64

In [145]:
# checking all rows where NO2_value column has Nan values
nan_NO2 = in_ulez_data[in_ulez_data['NO2_value'].isnull()]
nan_NO2.head()

# checking the months and how many values missing
print(nan_NO2.groupby(nan_NO2['date'].dt.strftime('%B'))['prov_rat'].count())
# checking the years and how many values missing
print(nan_NO2.groupby(nan_NO2['date'].dt.strftime('%Y'))['prov_rat'].count())
# checking which monitoring sites have nost missing values
print(nan_NO2.groupby(nan_NO2['site_name'])['prov_rat'].count())
# in_ulez_data.isna().sum()

date
April        146
August       184
December      94
February     106
January      154
July         189
June         172
March        150
May          164
November     141
October      151
September    158
Name: prov_rat, dtype: int64
date
2017    433
2018    267
2019    289
2020    375
2021    208
2022    237
Name: prov_rat, dtype: int64
site_name
Beech Street               97
Bloomsbury                119
Covent Garden             308
Elephant and Castle       115
Holborn (Bee Midtown)     465
Horseferry Road           178
Marylebone Road            72
Oxford Street             112
Strand (Northbank BID)    167
The Aldgate School         68
Walbrook Wharf            108
Name: prov_rat, dtype: int64


We have checked if it is possible to obtain NO2 data manually, but it seems like there is no data available. We will drop all the rows with Nan for NO2.

In [146]:
# dropping rows with Nan in NO2_value
in_ulez_data = in_ulez_data[in_ulez_data['NO2_value'].notna()]

in_ulez_data.isna().sum()

site_name                 0
site_borough              0
site_code                 0
latitude                  0
longitude                 0
site_type                 0
date                      0
NO2_value                 0
prov_rat                  0
avg_temperature           0
min_temperature          50
max_temperature          32
precipitation            39
snow_depth            20668
avg_wind_direction      107
avg_wind_speed           78
wind_gust              6463
avg_air_pressure         61
dtype: int64

In [147]:
# dropping wind_gust column
in_ulez_data = in_ulez_data.drop(['wind_gust'], axis=1)

# replacing Nan values in snow_depth column with 0
in_ulez_data['snow_depth'] = in_ulez_data['snow_depth'].fillna(0)

Now we have columns min_temperature, max_temperature, precipitation, avg_wind_direction, avg_wind_speed, avg_air_pressure remaining with Nan values. After careful consideration we decided to drop the rows with Nan.

In [148]:
in_ulez_data.isna().sum()

site_name               0
site_borough            0
site_code               0
latitude                0
longitude               0
site_type               0
date                    0
NO2_value               0
prov_rat                0
avg_temperature         0
min_temperature        50
max_temperature        32
precipitation          39
snow_depth              0
avg_wind_direction    107
avg_wind_speed         78
avg_air_pressure       61
dtype: int64

In [149]:
# dropping rows with Nan in min_temperature, max_temperature, precipitation, avg_wind_direction, avg_wind_speed, avg_air_pressure
in_ulez_data = in_ulez_data.dropna()

in_ulez_data.isna().sum()

site_name             0
site_borough          0
site_code             0
latitude              0
longitude             0
site_type             0
date                  0
NO2_value             0
prov_rat              0
avg_temperature       0
min_temperature       0
max_temperature       0
precipitation         0
snow_depth            0
avg_wind_direction    0
avg_wind_speed        0
avg_air_pressure      0
dtype: int64

In [150]:
in_ulez_data.shape

(21051, 17)

That is the final dataset we will work with from here:

In [151]:
in_ulez_data.head()

,site_name,site_borough,site_code,latitude,longitude,site_type,date,NO2_value,prov_rat,avg_temperature,min_temperature,max_temperature,precipitation,snow_depth,avg_wind_direction,avg_wind_speed,avg_air_pressure
0,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,2017-01-01,24.5,R,5.9,4.0,9.4,0.3,0.0,241.0,14.0,1018.6
1,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,2017-02-01,34.5,R,10.5,8.0,12.0,3.0,0.0,180.0,13.3,1006.5
2,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,2017-03-01,46.9,R,8.0,6.0,10.0,1.5,0.0,248.0,20.0,998.2
3,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,2017-04-01,45.8,R,12.8,11.0,15.0,0.3,0.0,212.0,15.6,1009.4
4,Bloomsbury,Camden,BL0,51.522287,-0.125848,Urban Background,2017-05-01,59.6,R,11.2,10.0,13.0,1.5,0.0,117.0,16.1,1004.2


In [152]:
# saving to csv file
in_ulez_data.to_csv('in_ulez_clean.csv',  index=False)

# **EXPANDED ULEZ**

We need to choose sites wich are relevant to us:

**Expanded ULEZ**

*Boroughs:*

1.   Barnet
2.   Brent
3.   Camden
4.   City of London
5.   Ealing
6.   Enfield
7.   Greenwich
8.   Hackney
9.   Hammersmith and Fulham
10.  Haringey
11.  Hounslow
12.  Islington
13.  Kensington and Chelsea
14.  Lambeth
15.  Lewisham
16.  Newham
17.  Redbridge
18.  Richmond upon Thames
19.  Southwark
20.  Tower Hamlets
21.  Waltham Forest
22.  Wandsworth
23.  Westminster

In [153]:
# choosing sites for expanded ULEZ
expanded_ulez_boroughs = ['Barnet', 'Brent', 'Camden', 'City of London', 'Ealing', 'Enfield', 'Greenwich', 'Hackney', 'Hammersmith and Fulham',
                          'Haringey', 'Hounslow', 'Islington', 'Kensington and Chelsea', 'Lambeth', 'Lewisham', 'Newham', 'Redbridge',
                          'Richmond', 'Southwark', 'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster']

exp_ulez_sites = sites[sites['borough'].isin(expanded_ulez_boroughs)]

exp_ulez_sites.borough.value_counts().sort_index()

Barnet                     3
Brent                      8
Camden                     8
City of London             8
Ealing                    14
Enfield                    7
Greenwich                 12
Hackney                    6
Hammersmith and Fulham     4
Haringey                   4
Hounslow                   9
Islington                  5
Kensington and Chelsea    10
Lambeth                    6
Lewisham                   7
Newham                     6
Redbridge                  6
Richmond                   5
Southwark                 10
Tower Hamlets              6
Waltham Forest             5
Wandsworth                11
Westminster               22
Name: borough, dtype: int64

We should find sites with:

latitude between 51.442633000000 and 51.614675000000

longitude between -0.292488000000 and 0.0707660000000

These are approximate coordinates of borders of the expanded ULEZ.

In [154]:
exp_ulez_sites = exp_ulez_sites.loc[(exp_ulez_sites['latitude'] > 51.442633000000) & (exp_ulez_sites['latitude'] <= 51.614675000000)]
exp_ulez_sites = exp_ulez_sites.loc[(exp_ulez_sites['longitude'] >= -0.292488000000) & (exp_ulez_sites['longitude'] < 0.0707660000000)]
exp_ulez_sites.shape

(149, 8)

In [155]:
# dropping sites with date_closed before 01/01/2023
# dropping sites with date_opened after 01/01/2017
open_code = pd.to_datetime('2017-01-01')
close_date = pd.to_datetime('2023-01-01')

exp_ulez_sites = exp_ulez_sites.loc[exp_ulez_sites['date_opened'] < open_code]
exp_ulez_sites = exp_ulez_sites.loc[exp_ulez_sites['date_closed'] > close_date]

# resetting the index
exp_ulez_sites = exp_ulez_sites.reset_index(drop=True)

exp_ulez_sites.borough.value_counts().sort_index()

Brent                     3
Camden                    3
City of London            4
Ealing                    4
Enfield                   1
Greenwich                 3
Hackney                   1
Haringey                  2
Islington                 2
Kensington and Chelsea    1
Lambeth                   2
Lewisham                  2
Newham                    2
Redbridge                 1
Richmond                  2
Southwark                 3
Tower Hamlets             2
Wandsworth                6
Westminster               6
Name: borough, dtype: int64

In [156]:
exp_ulez_sites.shape

(50, 8)

In [157]:
exp_ulez_sites.head()

,name,code,borough,type,latitude,longitude,date_opened,date_closed
0,Ikea,BT4,Brent,Roadside,51.552476,-0.258089,2003-06-20,2023-08-01 00:00:00
1,John Keble Primary School,BT6,Brent,Roadside,51.537799,-0.247793,2006-11-01,2023-08-01 00:00:00
2,Neasden Lane,BT5,Brent,Industrial,51.552656,-0.248774,2004-02-29,2023-08-01 00:00:00
3,Bloomsbury,BL0,Camden,Urban Background,51.522287,-0.125848,1992-01-23,2023-08-01 00:00:00
4,Holborn (Bee Midtown),IM1,Camden,Kerbside,51.517368,-0.120195,2014-02-06,2023-08-01 00:00:00


In [158]:
# creating dictionaries with code and columns name, borough, type, latitude, longitude
name_d = dict(zip(exp_ulez_sites.code, exp_ulez_sites.name))
borough_d = dict(zip(exp_ulez_sites.code, exp_ulez_sites.borough))
type_d = dict(zip(exp_ulez_sites.code, exp_ulez_sites.type))
latitude_d = dict(zip(exp_ulez_sites.code, exp_ulez_sites.latitude))
longitude_d = dict(zip(exp_ulez_sites.code, exp_ulez_sites.longitude))
print(name_d)
print(borough_d)
print(type_d)
print(latitude_d)
print(longitude_d)

{'BT4': 'Ikea', 'BT6': 'John Keble Primary School', 'BT5': 'Neasden Lane', 'BL0': 'Bloomsbury', 'IM1': 'Holborn (Bee Midtown)', 'CD1': 'Swiss Cottage', 'CT4': 'Beech Street', 'CT2': 'Farringdon Street', 'CT3': 'The Aldgate School', 'CT6': 'Walbrook Wharf', 'EA6': 'Hanger Lane Gyratory', 'EA8': 'Horn Lane', 'EI8': 'Horn Lane TEOM', 'EI1': 'Western Avenue', 'EN5': 'Bowes Primary School', 'GR7': 'Blackheath', 'GR9': 'Westhorne Avenue', 'GR8': 'Woolwich Flyover', 'HK6': 'Old Street', 'HG4': 'Priory Park South', 'HG1': 'Haringey Town Hall', 'IS6': 'Arsenal', 'IS2': 'Holloway Road', 'KC1': 'North Ken', 'LB5': 'Bondway Interchange', 'LB4': 'Brixton Road', 'LW4': 'Loampit Vale', 'LW2': 'New Cross', 'MR8': 'BAM', 'NM2': 'Cam Road', 'NM3': 'Wren Close', 'RB4': 'Gardner Close', 'RI2': 'Barnes Wetlands', 'RI1': 'Castelnau', 'SK5': 'A2 Old Kent Road', 'SK9': 'A2 Old Kent Road FIDAS', 'SK6': 'Elephant and Castle', 'TH4': 'Blackwall', 'TH2': 'Mile End Road', 'WAA': 'Battersea', 'WAC': 'Lavender Hill 

Now we need to get the names and the codes of the sites. We will use them to pull data about NO2 from London Air using API.

In [159]:
# getting sites' codes
exp_ulez_sites_codes = exp_ulez_sites.code
print(type(exp_ulez_sites_codes))

# converting pandas series to list
exp_ulez_sites_codes = exp_ulez_sites_codes.to_list()
print(exp_ulez_sites_codes)
print(type(exp_ulez_sites_codes))

<class 'pandas.core.series.Series'>
['BT4', 'BT6', 'BT5', 'BL0', 'IM1', 'CD1', 'CT4', 'CT2', 'CT3', 'CT6', 'EA6', 'EA8', 'EI8', 'EI1', 'EN5', 'GR7', 'GR9', 'GR8', 'HK6', 'HG4', 'HG1', 'IS6', 'IS2', 'KC1', 'LB5', 'LB4', 'LW4', 'LW2', 'MR8', 'NM2', 'NM3', 'RB4', 'RI2', 'RI1', 'SK5', 'SK9', 'SK6', 'TH4', 'TH2', 'WAA', 'WAC', 'WA9', 'WA7', 'WA8', 'WA2', 'WM5', 'WM0', 'MY1', 'WM6', 'NB1']
<class 'list'>


Below the Nitrogen Dioxide (NO2) data from 01/01/2017 to 01/01/2023 for the expanded ulez is imported.

In [160]:
exp_ulez_data = pd.read_csv('/content/drive/MyDrive/ULEZ/no2_exp_ulez.csv')
exp_ulez_data.head()

,Site,Species,ReadingDateTime,Value,Units,Provisional or Ratified
0,BT4,NO2,01/01/2017 00:00,60.8,ug m-3,R
1,BT4,NO2,02/01/2017 00:00,79.1,ug m-3,R
2,BT4,NO2,03/01/2017 00:00,96.5,ug m-3,R
3,BT4,NO2,04/01/2017 00:00,72.5,ug m-3,R
4,BT4,NO2,05/01/2017 00:00,136.0,ug m-3,R


In [161]:
exp_ulez_data.shape

(109550, 6)

In [162]:
# adding columns name, borough, type	latitude	longitude to exp_ulez_data
exp_ulez_data['name'] = exp_ulez_data['Site'].map(name_d)
exp_ulez_data['borough'] = exp_ulez_data['Site'].map(borough_d)
exp_ulez_data['type'] = exp_ulez_data['Site'].map(type_d)
exp_ulez_data['latitude'] = exp_ulez_data['Site'].map(latitude_d)
exp_ulez_data['longitude'] = exp_ulez_data['Site'].map(longitude_d)

In [163]:
exp_ulez_data.head()

,Site,Species,ReadingDateTime,Value,Units,Provisional or Ratified,name,borough,type,latitude,longitude
0,BT4,NO2,01/01/2017 00:00,60.8,ug m-3,R,Ikea,Brent,Roadside,51.552476,-0.258089
1,BT4,NO2,02/01/2017 00:00,79.1,ug m-3,R,Ikea,Brent,Roadside,51.552476,-0.258089
2,BT4,NO2,03/01/2017 00:00,96.5,ug m-3,R,Ikea,Brent,Roadside,51.552476,-0.258089
3,BT4,NO2,04/01/2017 00:00,72.5,ug m-3,R,Ikea,Brent,Roadside,51.552476,-0.258089
4,BT4,NO2,05/01/2017 00:00,136.0,ug m-3,R,Ikea,Brent,Roadside,51.552476,-0.258089


In [164]:
# making all columns lower case
exp_ulez_data.columns = exp_ulez_data.columns.str.lower()

# renaming columns
exp_ulez_data = exp_ulez_data.rename(columns={'readingdatetime': 'reading_date', 'provisional or ratified': 'prov_rat', 'site': 'code'})

# rearranging the columns in convinient way
columns = exp_ulez_data.columns.tolist()
new_order_columns = ['name', 'borough', 'code', 'latitude', 'longitude', 'type', 'species', 'reading_date', 'value', 'units', 'prov_rat']
exp_ulez_data = exp_ulez_data[new_order_columns]

# removing the time from date_closed and date_opened
exp_ulez_data['reading_date'] = pd.to_datetime(exp_ulez_data['reading_date'])
exp_ulez_data['reading_date'] = exp_ulez_data['reading_date'].dt.date

exp_ulez_data.head()

,name,borough,code,latitude,longitude,type,species,reading_date,value,units,prov_rat
0,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-01-01,60.8,ug m-3,R
1,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-02-01,79.1,ug m-3,R
2,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-03-01,96.5,ug m-3,R
3,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-04-01,72.5,ug m-3,R
4,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-05-01,136.0,ug m-3,R


In [165]:
exp_ulez_data['ID'] = exp_ulez_data['reading_date'].astype(str) + exp_ulez_data['latitude'].astype(str) + exp_ulez_data['longitude'].astype(str)

exp_ulez_data = exp_ulez_data.set_index('ID')

exp_ulez_data.head()

,name,borough,code,latitude,longitude,type,species,reading_date,value,units,prov_rat
ID,,,,,,,,,,,
2017-01-0151.552476-0.258089,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-01-01,60.8,ug m-3,R
2017-02-0151.552476-0.258089,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-02-01,79.1,ug m-3,R
2017-03-0151.552476-0.258089,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-03-01,96.5,ug m-3,R
2017-04-0151.552476-0.258089,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-04-01,72.5,ug m-3,R
2017-05-0151.552476-0.258089,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-05-01,136.0,ug m-3,R


In [166]:
exp_ulez_data.shape

(109550, 11)

# Collecting weather data for expanded area

In [167]:
# use columns from in_ulez_data to create dictionary of coordinates
columns_to_include = ['latitude', 'longitude']

# Convert selected columns to a dictionary
location_sites_exp = exp_ulez_sites[columns_to_include].to_dict('index')

location_sites_exp

{0: {'latitude': 51.552476, 'longitude': -0.258089},
 1: {'latitude': 51.537799, 'longitude': -0.247793},
 2: {'latitude': 51.552656, 'longitude': -0.248774},
 3: {'latitude': 51.522287, 'longitude': -0.125848},
 4: {'latitude': 51.5173675146177, 'longitude': -0.1201947113171},
 5: {'latitude': 51.544219, 'longitude': -0.175284},
 6: {'latitude': 51.520225273171, 'longitude': -0.0961060474176381},
 7: {'latitude': 51.5145253362314, 'longitude': -0.104515626337876},
 8: {'latitude': 51.513847178423, 'longitude': -0.077765681752},
 9: {'latitude': 51.510499, 'longitude': -0.091634},
 10: {'latitude': 51.53085, 'longitude': -0.292488},
 11: {'latitude': 51.518948, 'longitude': -0.265617},
 12: {'latitude': 51.5189160348806, 'longitude': -0.265652052386222},
 13: {'latitude': 51.5236078191481, 'longitude': -0.265502631799754},
 14: {'latitude': 51.613865, 'longitude': -0.125338},
 15: {'latitude': 51.472504, 'longitude': -0.012381},
 16: {'latitude': 51.456357, 'longitude': 0.040725},
 17:

In [168]:
def get_weather(x):
  # empty list for data
  weather_data_list = []
# for loop to iterate over the location sites dictionary created in the previous step
  for loc, coordinates in x.items():
    latitude = coordinates['latitude']
    longitude = coordinates['longitude']

 # combine the lat and lon and use Point to find the site
    location = Point(latitude, longitude)

 # specify date range
    start = datetime(2017, 1, 1)
    end = datetime(2023, 1, 1)

 # make the call
    weather_data = Daily(location, start, end).fetch()

# Add latitude and longitude columns to the weather data
    weather_data['latitude'] = latitude
    weather_data['longitude'] = longitude

# append the data to the empty list
    weather_data_list.append(weather_data)

# transform the data into a DataFrame
  combined_weather_data = pd.concat(weather_data_list)

  return combined_weather_data

# call the function
weather_df_exp = get_weather(location_sites_exp)

# print data to check
print(weather_df_exp.head())

            tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun  \
time                                                                        
2017-01-01   5.9   4.0   9.4   0.3   NaN  241.0  14.0   NaN  1018.6   NaN   
2017-01-02   1.9  -2.6   5.1   6.6   NaN  352.0  12.2   NaN  1028.4   NaN   
2017-01-03   0.9  -3.5   5.1   0.0   NaN  262.0  11.2   NaN  1028.0   NaN   
2017-01-04   3.9  -1.3   8.2   0.3   NaN  314.0  12.5   NaN  1025.6   NaN   
2017-01-05   0.0  -3.3   6.4   0.0   NaN  315.0   4.4   NaN  1034.4   NaN   

             latitude  longitude  
time                              
2017-01-01  51.552476  -0.258089  
2017-01-02  51.552476  -0.258089  
2017-01-03  51.552476  -0.258089  
2017-01-04  51.552476  -0.258089  
2017-01-05  51.552476  -0.258089  


In [169]:
weather_df_exp.shape

(109600, 12)

In [170]:
# Optional - Export the DataFrame to a TSV file
# file_path = "weather_data_expanded.tsv"
# weather_df.to_csv(file_path, sep='\t', index=False)

# print(f"DataFrame exported to {file_path}")

In [171]:
weather_df_exp.reset_index(inplace=True, level=['time'])

In [172]:
# renaming columns
weather_df_exp = weather_df_exp.rename(columns={'time': 'reading_date'})

print(weather_df_exp.head())

  reading_date  tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun  \
0   2017-01-01   5.9   4.0   9.4   0.3   NaN  241.0  14.0   NaN  1018.6   NaN   
1   2017-01-02   1.9  -2.6   5.1   6.6   NaN  352.0  12.2   NaN  1028.4   NaN   
2   2017-01-03   0.9  -3.5   5.1   0.0   NaN  262.0  11.2   NaN  1028.0   NaN   
3   2017-01-04   3.9  -1.3   8.2   0.3   NaN  314.0  12.5   NaN  1025.6   NaN   
4   2017-01-05   0.0  -3.3   6.4   0.0   NaN  315.0   4.4   NaN  1034.4   NaN   

    latitude  longitude  
0  51.552476  -0.258089  
1  51.552476  -0.258089  
2  51.552476  -0.258089  
3  51.552476  -0.258089  
4  51.552476  -0.258089  


In [173]:
# rearranging the columns in to position date and lat and lon on left
columns = weather_df_exp.columns.tolist()
print(columns)

['reading_date', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun', 'latitude', 'longitude']


In [174]:
new_order_columns = ['reading_date','latitude', 'longitude','tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun']
weather_df_exp = weather_df_exp[new_order_columns]
print(weather_df_exp.head())

  reading_date   latitude  longitude  tavg  tmin  tmax  prcp  snow   wdir  \
0   2017-01-01  51.552476  -0.258089   5.9   4.0   9.4   0.3   NaN  241.0   
1   2017-01-02  51.552476  -0.258089   1.9  -2.6   5.1   6.6   NaN  352.0   
2   2017-01-03  51.552476  -0.258089   0.9  -3.5   5.1   0.0   NaN  262.0   
3   2017-01-04  51.552476  -0.258089   3.9  -1.3   8.2   0.3   NaN  314.0   
4   2017-01-05  51.552476  -0.258089   0.0  -3.3   6.4   0.0   NaN  315.0   

   wspd  wpgt    pres  tsun  
0  14.0   NaN  1018.6   NaN  
1  12.2   NaN  1028.4   NaN  
2  11.2   NaN  1028.0   NaN  
3  12.5   NaN  1025.6   NaN  
4   4.4   NaN  1034.4   NaN  


In [175]:
weather_df_exp.shape

(109600, 13)

In [176]:
# create an ID of a unqiue value by combining date, lon and lat and coverting to string so they concat. Set it as index

weather_df_exp['ID'] = weather_df_exp['reading_date'].astype(str) + weather_df_exp['latitude'].astype(str) + weather_df_exp['longitude'].astype(str)

weather_df_exp = weather_df_exp.set_index('ID')

weather_df_exp.head()

,reading_date,latitude,longitude,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
ID,,,,,,,,,,,,,
2017-01-0151.552476-0.258089,2017-01-01,51.552476,-0.258089,5.9,4.0,9.4,0.3,NaN,241.0,14.0,NaN,1018.6,NaN
2017-01-0251.552476-0.258089,2017-01-02,51.552476,-0.258089,1.9,-2.6,5.1,6.6,NaN,352.0,12.2,NaN,1028.4,NaN
2017-01-0351.552476-0.258089,2017-01-03,51.552476,-0.258089,0.9,-3.5,5.1,0.0,NaN,262.0,11.2,NaN,1028.0,NaN
2017-01-0451.552476-0.258089,2017-01-04,51.552476,-0.258089,3.9,-1.3,8.2,0.3,NaN,314.0,12.5,NaN,1025.6,NaN
2017-01-0551.552476-0.258089,2017-01-05,51.552476,-0.258089,0.0,-3.3,6.4,0.0,NaN,315.0,4.4,NaN,1034.4,NaN


In [177]:
# merge the two dataframes on the ID

merged_data_ulez_2 = pd.merge(exp_ulez_data, weather_df_exp, on=['ID'])

merged_data_ulez_2.head()

,name,borough,code,latitude_x,longitude_x,type,species,reading_date_x,value,units,...,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
ID,,,,,,,,,,,,,,,,,,,,,
2017-01-0151.552476-0.258089,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-01-01,60.8,ug m-3,...,5.9,4.0,9.4,0.3,NaN,241.0,14.0,NaN,1018.6,NaN
2017-02-0151.552476-0.258089,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-02-01,79.1,ug m-3,...,10.4,9.3,11.2,3.0,NaN,180.0,14.6,NaN,1006.5,NaN
2017-03-0151.552476-0.258089,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-03-01,96.5,ug m-3,...,6.9,5.1,9.1,1.5,NaN,248.0,16.6,NaN,998.2,NaN
2017-04-0151.552476-0.258089,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-04-01,72.5,ug m-3,...,11.7,8.8,16.1,0.3,NaN,214.0,14.3,NaN,1009.4,NaN
2017-05-0151.552476-0.258089,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-05-01,136.0,ug m-3,...,10.7,8.4,13.3,1.5,NaN,109.0,14.2,NaN,1004.2,NaN


In [178]:
merged_data_ulez_2.shape

(113932, 24)

In [179]:
file_path = "merged_data_ulez_2.csv"

merged_data_ulez_2.to_csv(file_path)

print(f"DataFrame exported to {file_path}")

DataFrame exported to merged_data_ulez_2.csv


# Cleaning the second merged dataset

We repeated much of the same processes as above for the second dataset:

In [180]:
exp_ulez_data = pd.read_csv('/content/drive/MyDrive/ULEZ/merged_data_ulez_2.csv')
exp_ulez_data.head()

,ID,name,borough,code,latitude_x,longitude_x,type,species,reading_date_x,value,...,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2017-01-0151.552476-0.258089,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-01-01,60.8,...,5.9,4.0,9.4,0.3,NaN,241.0,14.0,NaN,1018.6,NaN
1,2017-02-0151.552476-0.258089,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-02-01,79.1,...,10.4,9.3,11.2,3.0,NaN,180.0,14.6,NaN,1006.5,NaN
2,2017-03-0151.552476-0.258089,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-03-01,96.5,...,6.9,5.1,9.1,1.5,NaN,248.0,16.6,NaN,998.2,NaN
3,2017-04-0151.552476-0.258089,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-04-01,72.5,...,11.7,8.8,16.1,0.3,NaN,214.0,14.3,NaN,1009.4,NaN
4,2017-05-0151.552476-0.258089,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,NO2,2017-05-01,136.0,...,10.7,8.4,13.3,1.5,NaN,109.0,14.2,NaN,1004.2,NaN


In [181]:
exp_ulez_data.shape

(113932, 25)

In [182]:
exp_ulez_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113932 entries, 0 to 113931
Data columns (total 25 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              113932 non-null  object 
 1   name            113932 non-null  object 
 2   borough         113932 non-null  object 
 3   code            113932 non-null  object 
 4   latitude_x      113932 non-null  float64
 5   longitude_x     113932 non-null  float64
 6   type            113932 non-null  object 
 7   species         113932 non-null  object 
 8   reading_date_x  113932 non-null  object 
 9   value           95090 non-null   float64
 10  units           113932 non-null  object 
 11  prov_rat        113932 non-null  object 
 12  reading_date_y  113932 non-null  object 
 13  latitude_y      113932 non-null  float64
 14  longitude_y     113932 non-null  float64
 15  tavg            113932 non-null  float64
 16  tmin            113672 non-null  float64
 17  tmax      

In [183]:
columns = exp_ulez_data.columns.tolist()
columns

['ID',
 'name',
 'borough',
 'code',
 'latitude_x',
 'longitude_x',
 'type',
 'species',
 'reading_date_x',
 'value',
 'units',
 'prov_rat',
 'reading_date_y',
 'latitude_y',
 'longitude_y',
 'tavg',
 'tmin',
 'tmax',
 'prcp',
 'snow',
 'wdir',
 'wspd',
 'wpgt',
 'pres',
 'tsun']

In [184]:
# checking if columns latitude_x and latitude_y, longitude_x and longitude_y, reading_date_x and reading_date_y are identical.
print(exp_ulez_data['latitude_x'].equals(exp_ulez_data['latitude_y']))
print(exp_ulez_data['longitude_x'].equals(exp_ulez_data['longitude_y']))
print(exp_ulez_data['reading_date_x'].equals(exp_ulez_data['reading_date_y']))

# the columns above are identical so we are dropping latitude_y, longitude_y and reading_date_y. Also dropping species column, because it has only one value which is NO2,
# ID - it was used for merging, and units, which also has the same value for all rows ug m-3, tsun - no values in there.
exp_ulez_data = exp_ulez_data.drop(['latitude_y', 'longitude_y', 'reading_date_y', 'species', 'ID', 'units', 'tsun'], axis=1)

# renaming columns
exp_ulez_data = exp_ulez_data.rename(columns={'name': 'site_name', 'borough': 'site_borough', 'code': 'site_code', 'type': 'site_type', 'latitude_x': 'latitude', 'longitude_x': 'longitude', 'reading_date_x': 'date', 'value': 'NO2_value', 'tavg': 'avg_temperature',
                                            'tmin': 'min_temperature', 'tmax': 'max_temperature', 'prcp': 'precipitation', 'snow': 'snow_depth', 'wdir': 'avg_wind_direction',
                                            'wspd': 'avg_wind_speed', 'wpgt': 'wind_gust', 'pres': 'avg_air_pressure'})

# converting the date format to datetime
exp_ulez_data['date'] = pd.to_datetime(exp_ulez_data['date'])

exp_ulez_data.head()

True
True
True


,site_name,site_borough,site_code,latitude,longitude,site_type,date,NO2_value,prov_rat,avg_temperature,min_temperature,max_temperature,precipitation,snow_depth,avg_wind_direction,avg_wind_speed,wind_gust,avg_air_pressure
0,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-01-01,60.8,R,5.9,4.0,9.4,0.3,NaN,241.0,14.0,NaN,1018.6
1,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-02-01,79.1,R,10.4,9.3,11.2,3.0,NaN,180.0,14.6,NaN,1006.5
2,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-03-01,96.5,R,6.9,5.1,9.1,1.5,NaN,248.0,16.6,NaN,998.2
3,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-04-01,72.5,R,11.7,8.8,16.1,0.3,NaN,214.0,14.3,NaN,1009.4
4,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-05-01,136.0,R,10.7,8.4,13.3,1.5,NaN,109.0,14.2,NaN,1004.2


In [185]:
exp_ulez_data.shape

(113932, 18)

In [186]:
exp_ulez_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113932 entries, 0 to 113931
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   site_name           113932 non-null  object        
 1   site_borough        113932 non-null  object        
 2   site_code           113932 non-null  object        
 3   latitude            113932 non-null  float64       
 4   longitude           113932 non-null  float64       
 5   site_type           113932 non-null  object        
 6   date                113932 non-null  datetime64[ns]
 7   NO2_value           95090 non-null   float64       
 8   prov_rat            113932 non-null  object        
 9   avg_temperature     113932 non-null  float64       
 10  min_temperature     113672 non-null  float64       
 11  max_temperature     113776 non-null  float64       
 12  precipitation       113574 non-null  float64       
 13  snow_depth          3588 non-

In [187]:
# checking all Nan values in the dataset
exp_ulez_data.isna().sum()

site_name                  0
site_borough               0
site_code                  0
latitude                   0
longitude                  0
site_type                  0
date                       0
NO2_value              18842
prov_rat                   0
avg_temperature            0
min_temperature          260
max_temperature          156
precipitation            358
snow_depth            110344
avg_wind_direction       654
avg_wind_speed           421
wind_gust              33852
avg_air_pressure         312
dtype: int64

In [188]:
# dropping rows with Nan in NO2_value, as with intial ULEZ
exp_ulez_data = exp_ulez_data[exp_ulez_data['NO2_value'].notna()]

exp_ulez_data.isna().sum()

site_name                 0
site_borough              0
site_code                 0
latitude                  0
longitude                 0
site_type                 0
date                      0
NO2_value                 0
prov_rat                  0
avg_temperature           0
min_temperature         221
max_temperature         137
precipitation           298
snow_depth            92252
avg_wind_direction      546
avg_wind_speed          360
wind_gust             28208
avg_air_pressure        265
dtype: int64

In [189]:
# dropping wind_gust column, as with intial ULEZ
exp_ulez_data = exp_ulez_data.drop(['wind_gust'], axis=1)

# replacing Nan values in snow_depth column with 0, as with intial ULEZ
exp_ulez_data['snow_depth'] = exp_ulez_data['snow_depth'].fillna(0)

In [190]:
exp_ulez_data.isna().sum()

site_name               0
site_borough            0
site_code               0
latitude                0
longitude               0
site_type               0
date                    0
NO2_value               0
prov_rat                0
avg_temperature         0
min_temperature       221
max_temperature       137
precipitation         298
snow_depth              0
avg_wind_direction    546
avg_wind_speed        360
avg_air_pressure      265
dtype: int64

In [191]:
# dropping rows with NaN, as with intial ULEZ
exp_ulez_data = exp_ulez_data.dropna()

exp_ulez_data.isna().sum()

site_name             0
site_borough          0
site_code             0
latitude              0
longitude             0
site_type             0
date                  0
NO2_value             0
prov_rat              0
avg_temperature       0
min_temperature       0
max_temperature       0
precipitation         0
snow_depth            0
avg_wind_direction    0
avg_wind_speed        0
avg_air_pressure      0
dtype: int64

In [192]:
exp_ulez_data.shape

(94246, 17)

In [193]:
exp_ulez_data.head()

,site_name,site_borough,site_code,latitude,longitude,site_type,date,NO2_value,prov_rat,avg_temperature,min_temperature,max_temperature,precipitation,snow_depth,avg_wind_direction,avg_wind_speed,avg_air_pressure
0,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-01-01,60.8,R,5.9,4.0,9.4,0.3,0.0,241.0,14.0,1018.6
1,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-02-01,79.1,R,10.4,9.3,11.2,3.0,0.0,180.0,14.6,1006.5
2,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-03-01,96.5,R,6.9,5.1,9.1,1.5,0.0,248.0,16.6,998.2
3,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-04-01,72.5,R,11.7,8.8,16.1,0.3,0.0,214.0,14.3,1009.4
4,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-05-01,136.0,R,10.7,8.4,13.3,1.5,0.0,109.0,14.2,1004.2


Our second cleaned dataset (sites from initial ULEZ + sites from expanded ULEZ):

In [194]:
exp_ulez_data.head()

,site_name,site_borough,site_code,latitude,longitude,site_type,date,NO2_value,prov_rat,avg_temperature,min_temperature,max_temperature,precipitation,snow_depth,avg_wind_direction,avg_wind_speed,avg_air_pressure
0,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-01-01,60.8,R,5.9,4.0,9.4,0.3,0.0,241.0,14.0,1018.6
1,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-02-01,79.1,R,10.4,9.3,11.2,3.0,0.0,180.0,14.6,1006.5
2,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-03-01,96.5,R,6.9,5.1,9.1,1.5,0.0,248.0,16.6,998.2
3,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-04-01,72.5,R,11.7,8.8,16.1,0.3,0.0,214.0,14.3,1009.4
4,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-05-01,136.0,R,10.7,8.4,13.3,1.5,0.0,109.0,14.2,1004.2


In [195]:
# saving to csv file
exp_ulez_data.to_csv('exp_ulez_clean.csv', index=False)

# EXPANDED SITES ONLY

We came back to this after we started visualising the data to see if having a file with only the expanded sites in would give us more insight:

In [196]:
# code to separate initial ulez sites from expanded
in_ulez_codes = ['BL0', 'IM1', 'CT4', 'CT2', 'CT3', 'CT6', 'MR8', 'SK6', 'WM5', 'WM0', 'MY1', 'WM6', 'NB1']

in_exp_ulez_codes = ['BT4', 'BT6', 'BT5', 'BL0', 'IM1', 'CD1', 'CT4', 'CT2', 'CT3', 'CT6', 'EA6', 'EA8', 'EI8', 'EI1',
                       'EN5', 'GR7', 'GR9', 'GR8', 'HK6', 'HG4', 'HG1', 'IS6', 'IS2', 'KC1', 'LB5', 'LB4', 'LW4', 'LW2',
                       'MR8', 'NM2', 'NM3', 'RB4', 'RI2', 'RI1', 'SK5', 'SK9', 'SK6', 'TH4', 'TH2', 'WAA', 'WAC', 'WA9',
                       'WA7', 'WA8', 'WA2', 'WM5', 'WM0', 'MY1', 'WM6', 'NB1']

expanded_ulez_codes = list(set(in_exp_ulez_codes) - set(in_ulez_codes))

expanded_ulez_codes.sort()

# only keeping rows with codes in expanded_ulez_codes
exp_ulez_data = exp_ulez_data[exp_ulez_data['site_code'].isin(expanded_ulez_codes)]

# checking if codes are correct
exp_ulez_data_site_codes = list(exp_ulez_data.site_code.unique())
exp_ulez_data_site_codes.sort()


print(expanded_ulez_codes)
print(exp_ulez_data_site_codes)

# saving to csv file
exp_ulez_data.to_csv('exp_ulez_sites_only_clean.csv', index=False)

['BT4', 'BT5', 'BT6', 'CD1', 'EA6', 'EA8', 'EI1', 'EI8', 'EN5', 'GR7', 'GR8', 'GR9', 'HG1', 'HG4', 'HK6', 'IS2', 'IS6', 'KC1', 'LB4', 'LB5', 'LW2', 'LW4', 'NM2', 'NM3', 'RB4', 'RI1', 'RI2', 'SK5', 'SK9', 'TH2', 'TH4', 'WA2', 'WA7', 'WA8', 'WA9', 'WAA', 'WAC']
['BT4', 'BT5', 'BT6', 'CD1', 'EA6', 'EA8', 'EI1', 'EN5', 'GR7', 'GR8', 'GR9', 'HG1', 'HG4', 'HK6', 'IS2', 'IS6', 'KC1', 'LB4', 'LB5', 'LW2', 'LW4', 'NM2', 'NM3', 'RB4', 'RI1', 'RI2', 'SK5', 'TH2', 'TH4', 'WA2', 'WA7', 'WA8', 'WA9', 'WAA', 'WAC']


In [197]:
exp_ulez_data.shape

(71946, 17)

In [198]:
exp_ulez_data.head()

,site_name,site_borough,site_code,latitude,longitude,site_type,date,NO2_value,prov_rat,avg_temperature,min_temperature,max_temperature,precipitation,snow_depth,avg_wind_direction,avg_wind_speed,avg_air_pressure
0,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-01-01,60.8,R,5.9,4.0,9.4,0.3,0.0,241.0,14.0,1018.6
1,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-02-01,79.1,R,10.4,9.3,11.2,3.0,0.0,180.0,14.6,1006.5
2,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-03-01,96.5,R,6.9,5.1,9.1,1.5,0.0,248.0,16.6,998.2
3,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-04-01,72.5,R,11.7,8.8,16.1,0.3,0.0,214.0,14.3,1009.4
4,Ikea,Brent,BT4,51.552476,-0.258089,Roadside,2017-05-01,136.0,R,10.7,8.4,13.3,1.5,0.0,109.0,14.2,1004.2
